# Microsoft Studios Movie Project


# 1. Introduction

## 1.1 Overview

Films are a major part of popular culture and a huge source of entertainment for many people. The market is projected to reach USD 409.02 billion by 2026 in terms of revenue. Therefore, it is wise to invest in this market. I will use data from various websites that contain information about the film industry.

## 1.2 Objectives

1. Business Understanding<br>
2. Data Understanding <br>
3. Data Preparation
4. Data Analysis
5. Conclusion

# 2. Business Understanding

The tech giant Microsoft has decided to venture into creating original video content and wants to establish a movie studio. My goal is to use exploratory data analysis to produce insights for Microsoft as they enter the film industry. I will be looking for answers to the following questions:

1.What are the most profitable genres, franchises, and stars in each market and how do they differ across regions?

2.How do critical and audience ratings and reviews affect the box office performance of different films and how can they be used to predict demand and optimize marketing strategies?

3.What are the current and future challenges and opportunities in the film industry and how can Microsoft leverage its strengths and resources to create original video content that meets the need of the users?


###2.1 Problem